In [1]:
from qgis.core import QgsApplication, QgsProcessingFeedback
from qgis.analysis import QgsNativeAlgorithms
import processing
import sys
import geopandas as gpd
from shapely.geometry import Point
from shapely import wkb

# Initialize QGIS Application
qgis_path = "C:/Program Files/QGIS 3.x/apps/qgis"
sys.path.append(qgis_path)
QgsApplication.setPrefixPath(qgis_path, True)
qgs = QgsApplication([], False)
qgs.initQgis()

# Add processing algorithms to registry
from processing.core.Processing import Processing
Processing.initialize()

# Define the GeoPackage path
gpkg = "C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Output\\output.gpkg"

# Remove third dimension
gdf_lines = gpd.read_file(gpkg, layer="drap")
_drop_z = lambda geom: wkb.loads(wkb.dumps(geom, output_dimension=2))
gdf_lines.geometry = gdf_lines.geometry.transform(_drop_z)
gdf_lines.to_file(gpkg, layer="drap_noZ", driver="GPKG")

In [2]:
# Define the paths for inputs and outputs
input_layer = "drap_noZ"
output_layer = "points_d25_off_10"

# Run Points along lines : every 25 meters, offset of 10 meters
processing.run("native:pointsalonglines", {
    'INPUT': f"{gpkg}|layername={input_layer}",
    'DISTANCE': 25,
    'START_OFFSET': 10,
    'END_OFFSET': 10,
    'OUTPUT': f'ogr:dbname=\'{gpkg}\' table="{output_layer}" (geom)'})

{'OUTPUT': 'C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Output\\output.gpkg|layername=points_d25_off_10'}

In [3]:
gdf_line = gpd.read_file(gpkg, layer="drap_noZ")
gdf_point_1 = gpd.read_file(gpkg, layer="points_d25_off_10")
gdf_point_2 = gpd.read_file(gpkg, layer="points_d25_off_10")

gdf_point_1.geometry = gdf_point_1.geometry.apply(lambda geom: Point(geom.x - 0.1, geom.y))
gdf_point_2.geometry = gdf_point_2.geometry.apply(lambda geom: Point(geom.x + 0.1, geom.y))

gdf_point_1.to_file(gpkg, layer="temp1", driver="GPKG")
gdf_point_2.to_file(gpkg, layer="temp2", driver="GPKG")

In [4]:
output_layer = "temp_lines"
temp1 = "temp1"
temp2 = "temp2"

# Create temporary lines from points
processing.run("native:hublines", 
               {'HUBS':f"{gpkg}|layername={temp1}",
                'HUB_FIELD':'fid',
                'HUB_FIELDS':[],
                'SPOKES':f"{gpkg}|layername={temp2}",
                'SPOKE_FIELD':'fid',
                'SPOKE_FIELDS':[],
                'GEODESIC':False,
                'GEODESIC_DISTANCE':1000,
                'ANTIMERIDIAN_SPLIT':False,
                'OUTPUT':f'ogr:dbname=\'{gpkg}\' table="{output_layer}" (geom)'})

{'OUTPUT': 'C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Output\\output.gpkg|layername=temp_lines'}

In [5]:
# Split lines
splited_layer = "splited_lines"
processing.run("native:splitwithlines",
               {'INPUT':f"{gpkg}|layername={input_layer}",
                'LINES':f"{gpkg}|layername={output_layer}",
                'OUTPUT':f'ogr:dbname=\'{gpkg}\' table="{splited_layer}" (geom)'})

{'OUTPUT': 'C:\\Users\\jperez\\Documents\\Current 1\\emc2\\Output\\output.gpkg|layername=splited_lines'}